In [21]:
import pandas as pd
import datetime

#api
import praw

#pre-processing
import nltk 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize
import re
# import string
#from textblob import TextBlob
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
# nltk.download()

from NLP_functions import display_topics, prob_df, topic_matrix

#analysis
from sklearn.decomposition import TruncatedSVD, NMF

In [4]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


#### Get top posts of all time

In [5]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

                                                 title  score      id  \
0    Elon Musk reminds me of a rich billionaire who...  39195  lce6zm   
1    CryptoNick is deleting all of his BitConnect v...  26505  7r0ftz   
2    Reminder: Robinhood blocked several stocks fro...  26228  lfrslo   
3    I will tell you exactly what is going on here,...  20146  7vga1y   
4    Robinhood is launching a Crypto Trading app to...  19966  7sx5ze   
..                                                 ...    ...     ...   
927  Apple Co-Founder Steve Wozniak Outlines the Pa...   1491  9rcezl   
928  PwC Hong Kong and PwC Singapore announce joint...   1489  8gy2l6   
929                                 We Are All Satoshi   1486  gqt4ze   
930  Hackers Donate Stolen Bitcoin Worth $20,000 to...   1487  jeowcu   
931             Top 101 Coins Grouped by Usage/Purpose   1486  lgeots   

          subreddit                                                url  \
0    CryptoCurrency  https://www.reddit.com/r/Cry

#### Get top coments from each top post

In [6]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        comment_string = ''
        for x in comments[:20]:
            comment_string = comment_string + ' ' + x
        top_comments.append([comment_string,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

                                               comment                date
0     If your life savings hinges on a Tweet from a... 2021-02-04 12:02:16
1     **Some Important Reminders:**\n\n* Even in th... 2018-01-17 11:46:08
2     It’s not even real Bitcoin.  You can never wi... 2021-02-09 01:55:15
3     [deleted] It's a shame i can vote this up onl... 2018-02-05 17:21:09
4     Reminder: all referral links are prohibited i... 2018-01-25 16:19:04
..                                                 ...                 ...
927   Not a bad supporter to have! Show me your bag... 2018-10-25 17:31:26
928   Alibaba working with PWC on a food supply cha... 2018-05-04 09:55:54
929   "I am the real Slim Shady" lol Except Craig W... 2020-05-26 08:28:25
930   Who is the true Robinhood here? > Bitcoin Wor... 2020-10-20 12:54:38
931   Well written! Didnt know there was soo many s... 2021-02-09 22:02:05

[932 rows x 2 columns]


## Cleaning

combine comments by year

In [9]:
top_comments['year'] = top_comments['date'].dt.year
years = ['2017', '2018', '2019', '2020', '2021']
top_comments['year'].value_counts()

2018    404
2020    200
2021    125
2019    114
2017     89
Name: year, dtype: int64

In [10]:
comment_year = pd.DataFrame(index=years, columns=['Comments'])
for y in years:
    comment_string = ''
    for index, row in top_comments.iterrows():
        if str(row[2]) == y:
            comment_string = comment_string + ' ' + row[0]
            
    comment_string = re.sub(r'\([^)]*\)', '', comment_string) #remove links
    comment_string = re.sub('\w*\d\w*', '', comment_string) #remove numbers
    comment_string = re.sub(r'[^\w\s]', '', (comment_string))#remove punctuation
    comment_string = comment_string.lower() #lowercase
    comment_string = comment_string.replace('btc', 'bitcoin') #replace bitcoin acronym 
    
    #create stop words list
    stop = stopwords.words('english')
    stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
            'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
    stop = set(stop)

    mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately')])
    
    comment_token = mwe_tokenizer.tokenize(word_tokenize(comment_string))  # tokenize words

    #lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    comment_token_lemm = [(lemmatizer.lemmatize(w)) for w in comment_token]

    comment_token_lemm = [w for w in comment_token_lemm if w not in stop] #remove stop words
    
    comment_token_lemm = " ".join(comment_token_lemm) #join back into one string
    
    comment_year.loc[y, 'Comments'] = comment_token_lemm #add to df

In [11]:
print(comment_year)

                                               Comments
2017  longer version raccoon loses two piece candyfl...
2018  important reminder even case public figure str...
2019  twitter post random guy popular wa giving legi...
2020  offered cabinet position biden admin tldr form...
2021  life saving hinge tweet crazy billionaire dont...


In [ ]:
# comment_year = re.sub(r'\([^)]*\)', '', all_comments) #remove links
# comment_year = re.sub('\w*\d\w*', '', all_comments) #remove numbers
# comment_year = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
# comment_year = comment_year.lower() #lowercase
# comment_year = comment_year.replace('btc', 'bitcoin') #replace bitcoin acronym 

In [ ]:
# all_comments = comment_year.loc['2017', 'Comments'] #concatenate comments column into one string

# all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links
# all_comments = re.sub('\w*\d\w*', '', all_comments) #remove numbers
# all_comments = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
# all_comments = all_comments.lower() #lowercase
# all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 

# #create stop words list
# stop = stopwords.words('english')
# stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
#         'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
# stop = set(stop)

# mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
#                              ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])

# words = mwe_tokenizer.tokenize(word_tokenize(all_comments))  # tokenize words

# #lemmatize words
# lemmatizer = nltk.stem.WordNetLemmatizer()
# words = [(lemmatizer.lemmatize(w)) for w in words]

# words = [w for w in words if w not in stop] #remove stop words

# #create n grams
# n = 2
# bigrams = ngrams(words, n)

# #count phrases
# counter = Counter()
# counter += Counter(bigrams)

# for word, count in counter.most_common(30):
#     print('%20s %i' % (" ".join(word), count))

In [ ]:
# new = s.apply(fun) 

# Topic Modeling

### Count Vectorization

In [12]:
comments = list(comment_year['Comments'])

cv = CountVectorizer(stop_words=stop)
X= cv.fit_transform(comments)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

topics = 3
topic_words = 10
topic_names = None

#### Latent Semantic Analysis (LSA) with Count Vectorization

In [14]:
lsa_cv = TruncatedSVD(topics)
X_lsa_cv = lsa_cv.fit_transform(X)
lsa_cv.explained_variance_ratio_

array([0.91733209, 0.03995111, 0.0229522 ])

In [22]:
print(topic_matrix(lsa_cv, cv.get_feature_names(), topics))
# topic_word = pd.DataFrame(lsa.components_.round(3),
#              index = ["component_1","component_2", "component_3"],
#              columns = cv.get_feature_names())
# topic_word

NameError: name 'pd' is not defined

In [20]:
# def display_topics(model, feature_names, no_top_words, topic_names=None):
#     for ix, topic in enumerate(model.components_):
#         if not topic_names or not topic_names[ix]:
#             print("\nTopic ", ix)
#         else:
#             print("\nTopic: '",topic_names[ix],"'")
#         print(", ".join([feature_names[i]
#                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa_cv, cv.get_feature_names(), topic_words, topic_names)

In [ ]:
prob_df(X_lsa_cv, topics)
# Vt = pd.DataFrame(doc_topic.round(5),
#              index = ['2017', '2018', '2019', '2020', '2021']s,
#              columns = ["component_1","component_2", "component_3" ])
# Vt

#### Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf_model = NMF(3)
X_topic = nmf_model.fit_transform(X)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, cv.get_feature_names(), 10)

In [ ]:
H = pd.DataFrame(doc_topic.round(5),
             index = comments,
             columns = ["component_1","component_2", "component_3" ])
H

## TF-ID Vectorizer

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
doc_vectors = vectorizer.fit_transform(documents)

In [23]:
jupyter kernelspec list

SyntaxError: invalid syntax (<ipython-input-23-68dcfb427789>, line 1)